<a href="https://colab.research.google.com/github/sjoseph25/data_2000/blob/main/LinearModels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
car_data_raw = pd.read_csv("https://cdn.c18l.org/vehicles_lab.csv")

I chose to keep the year, manufacturer, condition, fuel type, odometer, title status, and description. The purpose of year is to calculate the age of the car. Manufacturer, condition, title status, and odometer are some of the key players when looking at cars to buy. Fuel type is the potential wild card, since I am not sure whether it makes a big impact on car buying. My conjecture (based on 0 data) is that gas cars will be the cheapest of the fuel types. The purpose of description is to later on create the is_carvana column.

More conjectures based on no data would be that cars in excellent condition would be more expensive than the other conditions; the same goes for clean title status. Low odometers probably result in higher prices. I have no conjectures for manufacturer.

In [3]:
car_data = car_data_raw.drop(columns = ['region','model','cylinders','transmission','size',
                          'type', 'paint_color', 'state', 'lat', 'long', 'posting_date','drive'])
car_data.head()

,price,year,manufacturer,condition,fuel,odometer,title_status,description
0,15000,2013.0,ford,excellent,gas,128000.0,clean,2013 F-150 XLT V6 4 Door. Good condition. Leve...
1,27990,2012.0,gmc,good,gas,68696.0,clean,Carvana is the safer way to buy a car During t...
2,34590,2016.0,chevrolet,good,gas,29499.0,clean,Carvana is the safer way to buy a car During t...
3,35000,2019.0,toyota,excellent,gas,43000.0,clean,Selling my 2019 Toyota Tacoma TRD Off Road Dou...
4,29990,2016.0,chevrolet,good,gas,17302.0,clean,Carvana is the safer way to buy a car During t...


I one-hot encoded the condition, title status, and fuel because they were categorical. Manufacturer is in the section below this, but that categorical data was made into numerical as well (I did use your code from class and adjusted to this specific problem; thank you for the model).

I removed outliers from price, odometer, and year to remove values that would skew the data too far.

In [5]:
car_data = car_data.loc[car_data['price'] > 0]
car_data = car_data.loc[car_data['price'] <= 50000.0]

car_data = car_data.loc[car_data['odometer'] > 0]
car_data = car_data.loc[car_data['odometer'] <= 200000]

car_data = car_data.loc[car_data['year'] > 1975]

In [6]:
temp_cond = pd.get_dummies(car_data['condition'], dummy_na=True).astype(int)
temp_title = pd.get_dummies(car_data['title_status'], dummy_na=True).astype(int)
temp_fuel = pd.get_dummies(car_data['fuel'], dummy_na=True).astype(int)

In [7]:
new_car_data = pd.concat([car_data, temp_cond], axis = 1)

In [8]:
new_car_data = pd.concat([new_car_data, temp_title], axis = 1)

In [9]:
new_car_data = pd.concat([new_car_data, temp_fuel], axis = 1)

In [10]:
new_car_data.head()

,price,year,manufacturer,condition,fuel,odometer,title_status,description,excellent,fair,...,parts only,rebuilt,salvage,NaN,diesel,electric,gas,hybrid,other,NaN
0,15000,2013.0,ford,excellent,gas,128000.0,clean,2013 F-150 XLT V6 4 Door. Good condition. Leve...,1,0,...,0,0,0,0,0,0,1,0,0,0
1,27990,2012.0,gmc,good,gas,68696.0,clean,Carvana is the safer way to buy a car During t...,0,0,...,0,0,0,0,0,0,1,0,0,0
2,34590,2016.0,chevrolet,good,gas,29499.0,clean,Carvana is the safer way to buy a car During t...,0,0,...,0,0,0,0,0,0,1,0,0,0
3,35000,2019.0,toyota,excellent,gas,43000.0,clean,Selling my 2019 Toyota Tacoma TRD Off Road Dou...,1,0,...,0,0,0,0,0,0,1,0,0,0
4,29990,2016.0,chevrolet,good,gas,17302.0,clean,Carvana is the safer way to buy a car During t...,0,0,...,0,0,0,0,0,0,1,0,0,0


I chose to engineer the is_carvana column like we did in class because we saw that there was a difference in the prices of cars sold by Carvana and not sold by Carvana.

I chose to engineer the car age column because it offers a different, easier way of analyzing price vs year made because the program calculates the year of the car itself (rather than having to do it yourself -- it's more efficient).

In [12]:
#is_carvana
new_car_data['is_carvana'] = new_car_data['description'].str.contains('Carvana')
#car_age
new_car_data['year'] = new_car_data['year'].astype(int)

new_car_data['car_age'] = [2023] - new_car_data['year']
new_car_data = new_car_data[new_car_data['car_age'] >= 0]

new_car_data = new_car_data.drop(['year', 'condition', 'fuel','title_status'], axis=1)

In [13]:
manufacturers = {val: idx for idx, val in enumerate(new_car_data['manufacturer'].unique())}
new_car_data['manufacturer'] = new_car_data['manufacturer'].replace(manufacturers)

In [14]:
new_car_data['price'] = new_car_data['price'].astype(int)
new_car_data['odometer'] = new_car_data['odometer'].astype(int)

In [15]:
new_car_data.head()

,price,manufacturer,odometer,description,excellent,fair,good,like new,new,salvage,...,salvage,NaN,diesel,electric,gas,hybrid,other,NaN,is_carvana,car_age
0,15000,0,128000,2013 F-150 XLT V6 4 Door. Good condition. Leve...,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,False,10
1,27990,1,68696,Carvana is the safer way to buy a car During t...,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,True,11
2,34590,2,29499,Carvana is the safer way to buy a car During t...,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,True,7
3,35000,3,43000,Selling my 2019 Toyota Tacoma TRD Off Road Dou...,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,False,4
4,29990,2,17302,Carvana is the safer way to buy a car During t...,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,True,7


In [16]:
new_car_data.columns

Index([       'price', 'manufacturer',     'odometer',  'description',
          'excellent',         'fair',         'good',     'like new',
                'new',      'salvage',            nan,        'clean',
               'lien',      'missing',   'parts only',      'rebuilt',
            'salvage',            nan,       'diesel',     'electric',
                'gas',       'hybrid',        'other',            nan,
         'is_carvana',      'car_age'],
      dtype='object')

In [48]:
#bins
new_car_data['price_bin'] = pd.cut(x = new_car_data['price'],
    bins = np.linspace(0, 10000, 5+1), labels = False)
new_car_data.dropna(inplace = True)

In [49]:
new_car_data['price_bin'] = new_car_data['price_bin'].astype(int)

In [39]:
#training
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(new_car_data, train_size=0.2, random_state = 42)
model = LinearRegression().fit(
    X = train_data.loc[:, [
        'odometer', 'manufacturer', 'excellent',
        'good', 'fair', 'like new',
        'new', 'salvage', 'clean', 'rebuilt','salvage',
        'missing', 'lien','parts only', 'gas','diesel',
        'hybrid','electric', 'car_age', 'is_carvana']],
    y = train_data['price']
)

In [40]:
#score
model.score(
    X = test_data.loc[:, [
        'odometer', 'manufacturer', 'excellent',
        'good', 'fair', 'like new',
        'new', 'salvage', 'clean', 'rebuilt','salvage',
        'missing', 'lien','parts only', 'gas','diesel',
        'hybrid','electric', 'car_age', 'is_carvana']],
    y = test_data['price']
)

0.07849595619111283

In [50]:
#multinomial
from sklearn.linear_model import LogisticRegression
train_data, test_data = train_test_split(new_car_data, train_size=0.2, random_state = 42)
model2 = LogisticRegression(multi_class = 'multinomial', ).fit(
    y = train_data['price_bin'],
    X = train_data.loc[:, [
        'odometer', 'manufacturer', 'excellent',
        'good', 'fair', 'like new',
        'new', 'salvage', 'clean', 'rebuilt','salvage',
        'missing', 'lien','parts only', 'gas','diesel',
        'hybrid','electric', 'car_age', 'is_carvana']]
)

In [52]:
#testing score
model2.score(
    y = test_data['price_bin'],
    X = test_data.loc[:, [
        'odometer', 'manufacturer', 'excellent',
        'good', 'fair', 'like new',
        'new', 'salvage', 'clean', 'rebuilt','salvage',
        'missing', 'lien','parts only', 'gas','diesel',
        'hybrid','electric', 'car_age', 'is_carvana']],
)

0.23661433773055573

In [53]:
train_data.loc[:, [
        'price_bin', 'odometer', 'manufacturer', 'excellent',
        'good', 'fair', 'like new',
        'new', 'salvage', 'clean', 'rebuilt',
        'missing', 'lien','parts only', 'gas','diesel',
        'hybrid','electric', 'car_age', 'is_carvana']].corr()

,price_bin,odometer,manufacturer,excellent,good,fair,like new,new,salvage,salvage,...,rebuilt,missing,lien,parts only,gas,diesel,hybrid,electric,car_age,is_carvana
price_bin,1.000000,0.006129,-0.016579,0.149736,-0.004482,-0.201197,0.008147,-0.020208,-0.058352,-0.002841,...,0.061327,-0.054733,0.012497,-0.034203,-0.001788,-0.031193,0.060166,0.008205,-0.097316,-0.024019
odometer,0.006129,1.000000,-0.074196,-0.048441,0.156920,0.089170,-0.070663,-0.032959,-0.003451,-0.030604,...,-0.086530,0.003012,-0.019895,-0.012631,0.046800,-0.023644,0.003400,-0.042498,0.244676,-0.025334
manufacturer,-0.016579,-0.074196,1.000000,0.033977,-0.044941,-0.027002,0.047263,-0.009368,-0.014680,0.002603,...,0.004776,-0.016511,0.000032,-0.007876,0.065880,-0.026456,-0.053366,-0.011181,-0.041739,0.007783
excellent,0.149736,-0.048441,0.033977,1.000000,-0.518383,-0.171023,-0.219189,-0.039826,-0.045645,-0.037486,...,0.014907,-0.031849,-0.008557,-0.008469,0.049067,-0.052628,0.032048,0.010680,-0.094858,-0.001660
good,-0.004482,0.156920,-0.044941,-0.518383,1.000000,-0.142725,-0.182922,-0.033236,-0.038092,0.014369,...,0.012769,0.021102,0.013391,0.002112,0.045792,0.004805,-0.040091,-0.002659,0.190777,-0.032178
fair,-0.201197,0.089170,-0.027002,-0.171023,-0.142725,1.000000,-0.060349,-0.010965,-0.012567,0.039257,...,-0.004873,0.060206,0.024294,0.030590,-0.014022,0.058092,-0.019393,-0.007502,0.231409,-0.010616
like new,0.008147,-0.070663,0.047263,-0.219189,-0.182922,-0.060349,1.000000,-0.014053,-0.016107,-0.013063,...,0.008216,-0.013606,-0.003372,-0.008599,0.013168,-0.018704,0.016441,0.008242,-0.050344,-0.000971
new,-0.020208,-0.032959,-0.009368,-0.039826,-0.033236,-0.010965,-0.014053,1.000000,-0.002927,-0.006683,...,0.009446,-0.002472,-0.002709,-0.001562,-0.003823,-0.006793,0.005685,0.043982,-0.025825,-0.002472
salvage,-0.058352,-0.003451,-0.014680,-0.045645,-0.038092,-0.012567,-0.016107,-0.002927,1.000000,0.172679,...,-0.002224,0.081925,0.022693,0.087488,-0.000291,0.002658,0.003162,-0.002002,0.040408,-0.002833
salvage,-0.002841,-0.030604,0.002603,-0.037486,0.014369,0.039257,-0.013063,-0.006683,0.172679,1.000000,...,-0.023421,-0.006471,-0.007090,-0.004089,0.014697,-0.008504,-0.007755,-0.004573,-0.023208,-0.006471
